In [ ]:
import re 
import PyPDF2 as pdf2
import pandas as pd

In [ ]:
header = pdf2.PdfReader('RAZÃO DE JANEIRO A OUTUBRO 2023.pdf')

In [ ]:
for page in range(len(header.pages)):
    print(header.pages[page].extract_text())

In [ ]:
re.findall('VR. PGTO .+', header.pages[1].extract_text())

In [ ]:
fulltext = ''
for page in range(len(header.pages)):
    fulltext += header.pages[page].extract_text()+'\n'

In [ ]:

datacodigo = re.findall('\d\d/\d\d/\d\d\d\d \d.\d\d.\d\d\d', fulltext)
print(datacodigo)


In [ ]:
for linha in range(len(datacodigo)):
    datacodigo[linha] = datacodigo[linha].replace('\n', '')

In [ ]:
data = []
codigo = []

for linha in range(len(datacodigo)):
    if re.search('DA$', datacodigo[linha]):
        datacodigo[linha] += ' PECUARIA'
        
    data.append(re.search('\d\d/\d\d/\d\d\d\d', datacodigo[linha]).group())
    codigo.append(re.search('\d\.\d\d\.\d\d\d', datacodigo[linha]).group())

In [ ]:
print(len(data))
print(len(codigo))

In [ ]:
historicovalor = re.findall('Histórico: .+', fulltext)

In [ ]:
numerro = 0
saldo = []

for linha in range(len(historicovalor)):
    try: saldo.append(re.search(' -\d+\.?\d+\.?\d+,\d\d', historicovalor[linha]).group())
    
    except: 
        print(historicovalor[linha])
        print(linha)
        


In [ ]:
for item in range(len(saldo)):
    saldo[item] = saldo[item].replace(' ', '')
    saldo[item] = saldo[item].replace('.', '')
    saldo[item] = saldo[item].replace(',', '.')
    saldo[item] = float(saldo[item])

In [ ]:
valor = []

valor.append(str(round(saldo[0],2)))
valor[0] = valor[0].replace('-', '')
valor[0] = valor[0].replace('.', ',')

if not re.search('\,\d\d',valor[0]):
    valor[0] += '0'
    
if len(valor[0]) >= 7:
    valor[0] = valor[0][:-6] + '.' + valor[0][-6:] 
    
historico = []
historico.append(re.search('.+'+valor[0], historicovalor[0]).group().replace(valor[0], ''))
historico[0] = historico[0][11:-2]

for item in range(1, len(saldo)):
    valor.append(str(round(saldo[item] - saldo[item-1],2)))
    valor[item] = valor[item].replace('-', '')
    valor[item] = valor[item].replace('.', ',')
    
    if not re.search('\,\d\d',valor[item]):
        valor[item] += '0'
        
    if len(valor[item]) >= 7:
        valor[item] = valor[item][:-6] + '.' + valor[item][-6:] 
        
    try: 
        historico.append(re.search('.+'+valor[item], historicovalor[item]).group().replace(valor[item], '')[11:])
        
    except:
        print(historicovalor[item])
        print(valor[item])
        print(item)

In [ ]:
df = pd.DataFrame({'Data': data, 'Codigo' : codigo, 'Historico': historico, 'Valor': valor})
df.to_csv('out.csv', sep=';', encoding='utf-8-sig')